In [1]:
!pip install -qqq -U langchain_chroma
!pip install chromadb
!pip install -q -U FlagEmbedding
!pip install -qqq -U langchain-huggingface
!pip install -qqq -U langchain_core
!pip install -qqq -U langchain-community
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.1 MB/s eta 0:00:00


# Chroma Database

Chroma DB is a vector database system that allows you to store, retrieve, and manage embeddings. It can be used in Python or JavaScript with the chromadb library for local use, or connected to a remote server running Chroma. Users can configure Chroma to persist data on disk and create collections of embeddings using unique names.

In [2]:
import chromadb
from chromadb.config import Settings
from FlagEmbedding import FlagModel
import os
from langchain_core.documents import Document

In [3]:
model = FlagModel("BAAI/bge-large-en",
                  query_instruction_for_retrieval="Represent this sentence for searching relevant passages",
                  use_fp16=False)

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [4]:
sentences_1 = "Gujarat, located on the western coast of India, is rich in ancient history. Its settlements date back to the Indus Valley Civilization, with cities like Dholavira showing advanced urban planning. Over 4,000 years ago, it became a center of maritime trade. Gujarat played a crucial role in the spread of culture across the Arabian Sea, with links to Mesopotamia, the Persian Gulf, and even Africa."
sentences_2 = "The history of human settlement in Rajasthan goes back more than 100,000 years, with the Indus Valley Civilization marking one of the earliest urban settlements. Sites like Kalibangan show evidence of fire altars, and Rajasthan's strategic location made it a hub for trade. The region's culture and history have been shaped by its rich archaeological past, which continues to be explored."
sentences_3 = "Uttar Pradesh, a state in northern India, has a diverse economy dominated by agriculture, services, and manufacturing. The state is a significant contributor to India’s agriculture sector, producing crops like wheat, rice, and sugarcane. In recent years, the economy has shifted toward industrial growth, including sectors like textiles, electronics, and infrastructure development. The state is also a key player in India’s political landscape."

In [6]:
database_folder = 'database'
collection_name = 'genAI'

client = chromadb.PersistentClient(path=database_folder,settings=Settings(allow_reset=True))
client.reset()
collection = client.create_collection(name=collection_name)

In [8]:
collection.add(
    documents=[sentences_1, sentences_2, sentences_3],
    metadatas=[{"source": "Wikipedia"}, {"source": "google"}, {"source": "google"}],
    ids=["doc1", "doc2", "doc3"]
)
print("New collection created and documents added.")

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 104MiB/s]


New collection created and documents added.


In [9]:
results = collection.query(
    query_texts=[
        "Something about Gujarat"
    ],
    n_results=2
)

print(results)

{'ids': [['doc1', 'doc3']], 'embeddings': None, 'documents': [['Gujarat, located on the western coast of India, is rich in ancient history. Its settlements date back to the Indus Valley Civilization, with cities like Dholavira showing advanced urban planning. Over 4,000 years ago, it became a center of maritime trade. Gujarat played a crucial role in the spread of culture across the Arabian Sea, with links to Mesopotamia, the Persian Gulf, and even Africa.', 'Uttar Pradesh, a state in northern India, has a diverse economy dominated by agriculture, services, and manufacturing. The state is a significant contributor to India’s agriculture sector, producing crops like wheat, rice, and sugarcane. In recent years, the economy has shifted toward industrial growth, including sectors like textiles, electronics, and infrastructure development. The state is also a key player in India’s political landscape.']], 'uris': None, 'data': None, 'metadatas': [[{'source': 'Wikipedia'}, {'source': 'google

## Using Langchain chroma

### Adding texts




In [10]:
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

In [46]:
db_name = "bge_db"

if os.path.exists(db_name):
  Chroma(persist_directory = db_name,embedding_function = model).delete_collection()

texts = [sentences_1, sentences_2, sentences_3]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

chunks_per_text = [text_splitter.split_text(text) for text in texts]
flat_chunks = [chunk for text_chunks in chunks_per_text for chunk in text_chunks]

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en",encode_kwargs={'normalize_embeddings':True})
text_vectorstore = Chroma.from_texts(
    texts=flat_chunks,
    embedding=embeddings,
    persist_directory=db_name
)

print(f"Chroma Vectorstore created with {text_vectorstore._collection.count()} chunks")

Chroma Vectorstore created with 3 chunks


In [47]:
retriever_text = text_vectorstore.as_retriever(search_type="similarity",search_kwargs={"k": 2})

In [48]:
retriever_text.get_relevant_documents("Something about Gujarat")

[Document(metadata={}, page_content='Gujarat, located on the western coast of India, is rich in ancient history. Its settlements date back to the Indus Valley Civilization, with cities like Dholavira showing advanced urban planning. Over 4,000 years ago, it became a center of maritime trade. Gujarat played a crucial role in the spread of culture across the Arabian Sea, with links to Mesopotamia, the Persian Gulf, and even Africa.'),
 Document(metadata={}, page_content="The history of human settlement in Rajasthan goes back more than 100,000 years, with the Indus Valley Civilization marking one of the earliest urban settlements. Sites like Kalibangan show evidence of fire altars, and Rajasthan's strategic location made it a hub for trade. The region's culture and history have been shaped by its rich archaeological past, which continues to be explored.")]

### Adding documents

In [49]:
document_1 = Document(
    page_content=sentences_1,
    metadata={"source": "tweet"},
)
document_2 = Document(
    page_content=sentences_2,
    metadata={"source": "tweet"},
)
document_3 = Document(
    page_content=sentences_3,
    metadata={"source": "google"},
)

In [50]:
db_name = "bge_db_documents"

if os.path.exists(db_name):
  Chroma(persist_directory = db_name,embedding_function = model).delete_collection()

docs = [document_1, document_2,document_3]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

chunks=text_splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en",encode_kwargs={'normalize_embeddings':True})
vectorstore_documents = Chroma.from_documents(
    documents = chunks,
    embedding=embeddings,
    persist_directory=db_name
)

print(f"Chroma Vectorstore created with {vectorstore_documents._collection.count()} chunks")

Chroma Vectorstore created with 3 chunks


#### Using as retriever to search documents (Useful in Langchain)

In [51]:
retriever_doc = vectorstore_documents.as_retriever(search_type="similarity",search_kwargs={"k": 2})

In [52]:
retriever_doc.get_relevant_documents("Something about Gujarat")

[Document(metadata={'source': 'tweet'}, page_content='Gujarat, located on the western coast of India, is rich in ancient history. Its settlements date back to the Indus Valley Civilization, with cities like Dholavira showing advanced urban planning. Over 4,000 years ago, it became a center of maritime trade. Gujarat played a crucial role in the spread of culture across the Arabian Sea, with links to Mesopotamia, the Persian Gulf, and even Africa.'),
 Document(metadata={'source': 'tweet'}, page_content="The history of human settlement in Rajasthan goes back more than 100,000 years, with the Indus Valley Civilization marking one of the earliest urban settlements. Sites like Kalibangan show evidence of fire altars, and Rajasthan's strategic location made it a hub for trade. The region's culture and history have been shaped by its rich archaeological past, which continues to be explored.")]

#### Query directly on vectorstore

In [53]:
vectorstore_documents.similarity_search(
    "Uttar Pradesh",
    k=2,
)

[Document(metadata={'source': 'google'}, page_content='Uttar Pradesh, a state in northern India, has a diverse economy dominated by agriculture, services, and manufacturing. The state is a significant contributor to India’s agriculture sector, producing crops like wheat, rice, and sugarcane. In recent years, the economy has shifted toward industrial growth, including sectors like textiles, electronics, and infrastructure development. The state is also a key player in India’s political landscape.'),
 Document(metadata={'source': 'tweet'}, page_content="The history of human settlement in Rajasthan goes back more than 100,000 years, with the Indus Valley Civilization marking one of the earliest urban settlements. Sites like Kalibangan show evidence of fire altars, and Rajasthan's strategic location made it a hub for trade. The region's culture and history have been shaped by its rich archaeological past, which continues to be explored.")]

### Similarity search with score

In [54]:
results = vectorstore_documents.similarity_search_with_score(
    "Uttar Pradesh", k=1
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.303897] Uttar Pradesh, a state in northern India, has a diverse economy dominated by agriculture, services, and manufacturing. The state is a significant contributor to India’s agriculture sector, producing crops like wheat, rice, and sugarcane. In recent years, the economy has shifted toward industrial growth, including sectors like textiles, electronics, and infrastructure development. The state is also a key player in India’s political landscape. [{'source': 'google'}]


# Facebook AI Similarity Search (Faiss) Database

 FAISS is an open-source library built for similarity search and clustering of dense vectors. Faiss can be used to build an index and perform searches with remarkable speed and memory efficiency. Additionally, it enhances search performance through its GPU implementations for various indexing methods.

In [23]:
from langchain.vectorstores import FAISS

### Adding texts

In [55]:
faiss_text_vectorstore = FAISS.from_texts(flat_chunks, embeddings) # Creates an in memory docstore

In [56]:
faiss_text_retriever =faiss_text_vectorstore.as_retriever(search_type="similarity",search_kwargs={"k": 2})

In [57]:
faiss_text_retriever.get_relevant_documents("Something about Rajasthan")

[Document(id='9b62fa0a-46b8-4a95-b3aa-cbd68bfcae8d', metadata={}, page_content="The history of human settlement in Rajasthan goes back more than 100,000 years, with the Indus Valley Civilization marking one of the earliest urban settlements. Sites like Kalibangan show evidence of fire altars, and Rajasthan's strategic location made it a hub for trade. The region's culture and history have been shaped by its rich archaeological past, which continues to be explored."),
 Document(id='a6868d91-0020-473a-bb61-17cb9e9a828a', metadata={}, page_content='Gujarat, located on the western coast of India, is rich in ancient history. Its settlements date back to the Indus Valley Civilization, with cities like Dholavira showing advanced urban planning. Over 4,000 years ago, it became a center of maritime trade. Gujarat played a crucial role in the spread of culture across the Arabian Sea, with links to Mesopotamia, the Persian Gulf, and even Africa.')]

### Adding documents

In [29]:
faiss_vectorstore_doc = FAISS.from_documents(documents= chunks, embedding=embeddings)

#### Using as retriever to search documents (Useful in Langchain)

In [58]:
faiss_retriever_doc =faiss_vectorstore_doc.as_retriever(search_type="similarity",search_kwargs={"k": 2})

In [59]:
faiss_retriever_doc.get_relevant_documents("Something about Uttar Pradesh")

[Document(id='223511ad-1444-4848-90be-749d090df190', metadata={'source': 'google'}, page_content='Uttar Pradesh, a state in northern India, has a diverse economy dominated by agriculture, services, and manufacturing. The state is a significant contributor to India’s agriculture sector, producing crops like wheat, rice, and sugarcane. In recent years, the economy has shifted toward industrial growth, including sectors like textiles, electronics, and infrastructure development. The state is also a key player in India’s political landscape.'),
 Document(id='39d552ce-f8f5-4f3c-ab70-da9ee469c72f', metadata={'source': 'tweet'}, page_content="The history of human settlement in Rajasthan goes back more than 100,000 years, with the Indus Valley Civilization marking one of the earliest urban settlements. Sites like Kalibangan show evidence of fire altars, and Rajasthan's strategic location made it a hub for trade. The region's culture and history have been shaped by its rich archaeological past,

#### Query directly on vectorstore

In [60]:
faiss_vectorstore_doc.similarity_search(
    "Uttar Pradesh",
    k=2,
    # filter={"source": "tweet"}, # optional params
)

[Document(id='223511ad-1444-4848-90be-749d090df190', metadata={'source': 'google'}, page_content='Uttar Pradesh, a state in northern India, has a diverse economy dominated by agriculture, services, and manufacturing. The state is a significant contributor to India’s agriculture sector, producing crops like wheat, rice, and sugarcane. In recent years, the economy has shifted toward industrial growth, including sectors like textiles, electronics, and infrastructure development. The state is also a key player in India’s political landscape.'),
 Document(id='39d552ce-f8f5-4f3c-ab70-da9ee469c72f', metadata={'source': 'tweet'}, page_content="The history of human settlement in Rajasthan goes back more than 100,000 years, with the Indus Valley Civilization marking one of the earliest urban settlements. Sites like Kalibangan show evidence of fire altars, and Rajasthan's strategic location made it a hub for trade. The region's culture and history have been shaped by its rich archaeological past,

### Similarity search with score

In [61]:
results = faiss_vectorstore_doc.similarity_search_with_score(
    "Uttar Pradesh", k=1
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.303897] Uttar Pradesh, a state in northern India, has a diverse economy dominated by agriculture, services, and manufacturing. The state is a significant contributor to India’s agriculture sector, producing crops like wheat, rice, and sugarcane. In recent years, the economy has shifted toward industrial growth, including sectors like textiles, electronics, and infrastructure development. The state is also a key player in India’s political landscape. [{'source': 'google'}]


### Saving and loading

* We can also save and load a FAISS index. This is useful so we don't have to recreate it everytime you use it.

In [33]:
faiss_vectorstore_doc.save_local("faiss_index")

new_vector_store = FAISS.load_local(
    "faiss_index", embeddings, allow_dangerous_deserialization=True
)

docs = new_vector_store.similarity_search(" Uttar Pradesh")

In [34]:
docs

[Document(id='223511ad-1444-4848-90be-749d090df190', metadata={'source': 'google'}, page_content='Uttar Pradesh, a state in northern India, has a diverse economy dominated by agriculture, services, and manufacturing. The state is a significant contributor to India’s agriculture sector, producing crops like wheat, rice, and sugarcane. In recent years, the economy has shifted toward industrial growth, including sectors like textiles, electronics, and infrastructure development. The state is also a key player in India’s political landscape.'),
 Document(id='39d552ce-f8f5-4f3c-ab70-da9ee469c72f', metadata={'source': 'tweet'}, page_content="The history of human settlement in Rajasthan goes back more than 100,000 years, with the Indus Valley Civilization marking one of the earliest urban settlements. Sites like Kalibangan show evidence of fire altars, and Rajasthan's strategic location made it a hub for trade. The region's culture and history have been shaped by its rich archaeological past,